In [1]:
# import
import pyspark
from pyspark.sql import Row
import os
import json
import matplotlib.pyplot as plt
import numpy as np

In [2]:
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
.builder\
.master('local')\
.appName('myApp')\
.config(conf=myConf)\
.getOrCreate()

21/11/09 15:56:34 WARN Utils: Your hostname, Kritiasui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
21/11/09 15:56:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/09 15:56:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# 1번 정답

In [19]:
myData = []
f = open('myText.txt', 'r')
for line in f.readlines():
    myData.append([line.strip()])
myDf = spark.createDataFrame(myData, ['article'])
myDf.show()

+----------------------------------+
|                           article|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|         자랑스러운 경찰관 여러분,|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
| 정부는 누구도 법 위에 군림하지...|
| 경찰헌장은 따뜻한 경찰, 의로운...|
|행사를 준비하신 민갑룡 경찰청장...|
+----------------------------------+
only showing top 20 rows



# 2번 정답 (전문에서 추출, DF에서 추출)

In [18]:
words = []
for line in myData:
    words.append(line[0].split(' '))
words

[['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.'],
 ['국민의',
  '안전을',
  '위해',
  '밤낮없이',
  '애쓰시는',
  '전국의',
  '15만',
  '경찰관',
  '여러분께',
  '먼저',
  '감사를',
  '드립니다.',
  '전몰·순직',
  '경찰관들의',
  '고귀한',
  '희생에',
  '경의를',
  '표합니다.',
  '유가족',
  '여러분께',
  '위로의',
  '마음을',
  '전합니다.'],
 ['오늘',
  '홍조근정훈장을',
  '받으신',
  '중앙경찰학교장',
  '이은정',
  '치안감님,',
  '근정포장을',
  '받으신',
  '광주남부경찰서',
  '김동현',
  '경감님을',
  '비롯한',
  '수상자',
  '여러분께',
  '각별한',
  '축하와',
  '감사를',
  '드립니다.',
  '또한',
  '경찰',
  '영웅으로',
  '추서되신',
  '차일혁,',
  '최중락님께',
  '국민의',
  '사랑을',
  '전해드립니다.'],
 ['사랑하는', '경찰관', '여러분,'],
 ['여러분의',
  '헌신적',
  '노력으로',
  '우리의',
  '치안은',
  '참',
  '좋아졌습니다.',
  '지난해',
  '범죄',
  '발생은',
  '2015년에',
  '비해',
  '15.1%',
  '줄었습니다.',
  '같은',
  '기간',
  '교통사고',
  '사망자는',
  '18.2%',
  '감소했습니다.'],
 ['치안의',
  '개선은',
  '국민의',
  '체감으로',
  '나타나고',
  '있습니다.',
  '올해',
  '상반기',
  '국민의',
  '체감안전도는',
  '74.5점으로',
  '역대',
  '최고를',
  '기록했습니다.',
  '범죄안전도는',
  '처음으로',
  '80점을',
  '넘었습니다.'],
 ['한국을',
  '찾는',
  '외국',
  '관광객들도

In [16]:
from pyspark.ml.feature import RegexTokenizer
re = RegexTokenizer(inputCol="article", outputCol="wordsReg", pattern="\\s+")
reDf = re.transform(myDf)
reDf.select([reDf.wordsReg]).show()

+--------------------------------+
|                        wordsReg|
+--------------------------------+
|   [존경하는, 국민, 여러분,, ...|
|  [국민의, 안전을, 위해, 밤낮...|
|[오늘, 홍조근정훈장을, 받으신...|
|     [사랑하는, 경찰관, 여러분,]|
| [여러분의, 헌신적, 노력으로,...|
|  [치안의, 개선은, 국민의, 체...|
|  [한국을, 찾는, 외국, 관광객...|
|    [올해는, ‘경찰의, 날’에, ...|
|   [자랑스러운, 경찰관, 여러분,]|
| [경찰헌장은, “나라와, 겨레를...|
|    [대한민국, 경찰은, 1945년...|
|  [임시정부, 초대, 경무국장, ...|
|    [광복, 이후, 6·25전쟁에서...|
|  [그러나, 잘못도, 없지는, 않...|
|  [지금, 경찰은, 과거를, 돌아...|
|  [검경, 수사권, 조정과, 자치...|
| [정부는, 경찰의, 근무여건을,...|
|   [정부는, 누구도, 법, 위에,...|
|  [경찰헌장은, 따뜻한, 경찰,,...|
|  [행사를, 준비하신, 민갑룡, ...|
+--------------------------------+
only showing top 20 rows



# 3번 정답

In [29]:
from pyspark.ml.feature import StopWordsRemover
stopwords = []
for line in words:
    for word in line:
        if len(word) == 1:
            stopwords.append(word)
stopwords = list(set(stopwords))
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")
stop.setStopWords(stopwords)
stop.getStopWords()

['큰', '참', '등', '더', '수', '그', '법', '될', '돌']

# 4번 정답

In [31]:
stopDf=stop.transform(reDf)
stopDf.select(stopDf['nostops']).show()


+--------------------------------+
|                         nostops|
+--------------------------------+
|   [존경하는, 국민, 여러분,, ...|
|  [국민의, 안전을, 위해, 밤낮...|
|[오늘, 홍조근정훈장을, 받으신...|
|     [사랑하는, 경찰관, 여러분,]|
| [여러분의, 헌신적, 노력으로,...|
|  [치안의, 개선은, 국민의, 체...|
|  [한국을, 찾는, 외국, 관광객...|
|    [올해는, ‘경찰의, 날’에, ...|
|   [자랑스러운, 경찰관, 여러분,]|
| [경찰헌장은, “나라와, 겨레를...|
|    [대한민국, 경찰은, 1945년...|
|  [임시정부, 초대, 경무국장, ...|
|    [광복, 이후, 6·25전쟁에서...|
|  [그러나, 잘못도, 없지는, 않...|
|  [지금, 경찰은, 과거를, 돌아...|
|  [검경, 수사권, 조정과, 자치...|
| [정부는, 경찰의, 근무여건을,...|
|  [정부는, 누구도, 위에, 군림...|
|  [경찰헌장은, 따뜻한, 경찰,,...|
|  [행사를, 준비하신, 민갑룡, ...|
+--------------------------------+
only showing top 20 rows



# 5번 정답

In [79]:
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer

doc = reduce(lambda x,y: x+y, myData)
vectorizer = TfidfVectorizer(max_df=1.0, stop_words='english',norm = None)
vectorizer.fit_transform(doc)
vectorizer.vocabulary_

{'존경하는': 278,
 '국민': 80,
 '여러분': 212,
 '경찰관': 45,
 '일흔네': 251,
 '경찰의': 53,
 '입니다': 257,
 '국민의': 84,
 '안전을': 201,
 '위해': 232,
 '밤낮없이': 165,
 '애쓰시는': 206,
 '전국의': 266,
 '15만': 1,
 '여러분께': 213,
 '먼저': 144,
 '감사를': 15,
 '드립니다': 134,
 '전몰': 267,
 '순직': 194,
 '경찰관들의': 46,
 '고귀한': 59,
 '희생에': 346,
 '경의를': 42,
 '표합니다': 325,
 '유가족': 233,
 '위로의': 228,
 '마음을': 139,
 '전합니다': 270,
 '오늘': 220,
 '홍조근정훈장을': 340,
 '받으신': 159,
 '중앙경찰학교장': 290,
 '이은정': 243,
 '치안감님': 315,
 '근정포장을': 96,
 '광주남부경찰서': 75,
 '김동현': 103,
 '경감님을': 38,
 '비롯한': 175,
 '수상자': 192,
 '각별한': 14,
 '축하와': 309,
 '또한': 137,
 '경찰': 43,
 '영웅으로': 219,
 '추서되신': 308,
 '차일혁': 300,
 '최중락님께': 307,
 '사랑을': 178,
 '전해드립니다': 271,
 '사랑하는': 179,
 '여러분의': 215,
 '헌신적': 338,
 '노력으로': 112,
 '우리의': 227,
 '치안은': 317,
 '좋아졌습니다': 280,
 '지난해': 292,
 '범죄': 167,
 '발생은': 161,
 '2015년에': 5,
 '비해': 176,
 '15': 0,
 '줄었습니다': 288,
 '같은': 21,
 '기간': 97,
 '교통사고': 76,
 '사망자는': 180,
 '18': 2,
 '감소했습니다': 18,
 '치안의': 319,
 '개선은': 23,
 '체감으로': 304,
 '나타나고': 108,
 '있습니다': 259,
 

# 6번 정답

In [73]:
from pyspark.ml.feature import HashingTF, IDF
hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)
hashDf.select("nostops", "hash").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                                  

21/11/09 17:38:39 ERROR RetryingBlockFetcher: Exception while beginning fetch of 1 outstanding blocks (after 3 retries)
java.io.IOException: Failed to connect to /192.168.0.3:65397
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:287)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:218)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:123)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.fetchAllOutstanding(RetryingBlockFetcher.java:153)
	at org.apache.spark.network.shuffle.RetryingBlockFetcher.lambda$initiateRetry$0(RetryingBlockFetcher.java:181)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Wo